# **CureMate**

###Importar bibliotecas importantes

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 
import joblib
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
import pandas as pd

Mounted at /content/drive


In [ ]:
model = joblib.load("/content/drive/Shareddrives/OncoAI/Grupo 5 - OncoAI/Projeto/Base_de_dados/StackingClassifier.proj")
data = pd.read_csv("/content/drive/Shareddrives/OncoAI/Grupo 5 - OncoAI/Projeto/Base_de_dados/DataModel.csv")

###Selecione os valores de entrada para o modelo (confirme se o valor adicionado está na lista de valores possíveis)

Este é o modelo criado pelo grupo OncoAI. O modelo possui 8 valores de entrada:
*   A idade durante o diagnóstico primário
*   O subtipo tumoral
*   A leitura do receptor de estrogenio
*   A leitura do receptor de progesterona
*   A porcentagem encontrada de Ki67
*   O estadio clínico
*   A classificação TNM-T do paciente
*   A classificação TNM-N do paciente


Para fazer uma predição, basta:
1.   Insira os dados correspondentes nas variáveis abaixo
2.   Pressione Crtl + F9 para executar as células de código
3.   Espere o programa rodar e a resposta será dada após a última célula de código for executada

Caso o programa retone "Valor inserido inválido ou faltando", revise os dados inseridos e certifique-se de que os valores inseridos estão nos valores possíveis de cada variável




In [ ]:
idade_no_primeiro_diagnostico = 58 
#Valores possíveis: números naturais

subtipo_tumoral = 2 
#Valores possíveis: números naturais de 1 a 5

receptor_de_estrogenio = "positivo"  
#Valores possíveis: 'positivo','negativo', 'Nao realizado', 'inconclusivo'

receptor_de_progesterona = "negativo"  
#Valores possíveis: 'positivo','negativo', 'Nao realizado', 'inconclusivo'

ki67_percentage = 20  
#Valores possíveis: números reais entre 0 e 100

estadio_clinico = "IIIA"
#Valores possíveis: 'IIA', 'IIIA', 'IIB', 'IV', 'IIIB', 'IIIC', 'I','IA','0', 'X: não foi possível determinar', 'III', 'Y: NA': 5, 
#'II', 'IB', 'IVB', 'IVA', 'IVC'

classificacao_tnm_clinico_t = "3" 
#Valores possíveis: "2", "3", "1C", "4B", "4D", "1", "4", "1B", "1A", "4C", "4A", "IS", "CDIS", "3A"

classificacao_tnm_clinico_n = "1"
#Valores possíveis: "1", "2", "0", "3A", "2A", "3", "2B", "3B", "3C"

In [ ]:
input = pd.DataFrame(data={"idade_no_primeiro_diagnostico":[idade_no_primeiro_diagnostico],
                           "subtipo_tumoral":[subtipo_tumoral],
                           "receptor_de_estrogenio":[receptor_de_estrogenio],
                           "receptor_de_progesterona":[receptor_de_progesterona],
                           "ki67_percentage":[ki67_percentage],
                           "estadio_clinico":[estadio_clinico],
                           "classificacao_tnm_clinico_t":[classificacao_tnm_clinico_t],
                           "classificacao_tnm_clinico_n":[classificacao_tnm_clinico_n]})

###Tratamento dos dados inseridos

In [ ]:
input['receptor_de_estrogenio'] = input['receptor_de_estrogenio'].replace( { 
    'positivo': 1, 
    'Nao realizado': 2, 
    'negativo': 0 
  } )
input['receptor_de_progesterona'] = input['receptor_de_progesterona'].replace( { 
    'positivo': 1, 
    'negativo': 0, 
    'Nao realizado': 3, 
    'inconclusivo': 2 
  } )
input['estadio_clinico'] = input['estadio_clinico'].replace( { 
    'IIA': 2, 
    'IIIA': 3, 
    'IIB': 2, 
    'IV': 4, 
    'IIIB': 3, 
    'IIIC': 3, 
    'I': 1, 
    'IA': 1, 
    '0': 0,
    'X: não foi possível determinar': 6, 
    'III': 3, 
    'Y: NA': 5, 
    'II': 2, 
    'IB': 1, 
    'IVB': 4, 
    'IVA': 4,
    'IVC': 4
  } )
input["classificacao_tnm_clinico_t"] = input["classificacao_tnm_clinico_t"].replace({"2":2,
                                                                                   "3":3,
                                                                                   "1C":1,
                                                                                   "4B":4,
                                                                                   "4D":4,
                                                                                   "1":1,
                                                                                   "4":4,
                                                                                   "1B":1,
                                                                                   "1A":1,
                                                                                   "4C":4,
                                                                                   "4A":4,
                                                                                   "IS":0,
                                                                                   "CDIS":0,
                                                                                   "3A":3})
input["classificacao_tnm_clinico_n"] = input["classificacao_tnm_clinico_n"].replace({"1":1,
                                                                                   "2":2,
                                                                                   "0":0,
                                                                                   "3A":3,
                                                                                   "2A":2,
                                                                                   "3":3,
                                                                                   "2B":2,
                                                                                   "3B":3,
                                                                                   "3C":3})

In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

X = data.drop(columns=["regime_tratamento","Unnamed: 0"])
y = data["regime_tratamento"]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.2,
                                                    random_state = 50)

# Rebalanceando os dados com oversampling
ros = RandomOverSampler(random_state=0)
X, y = ros.fit_resample(X, y)

model.fit(X,y)

StackingClassifier(estimators=[('ada', AdaBoostClassifier(random_state=2186)),
                               ('lr',
                                LogisticRegression(max_iter=1000,
                                                   random_state=6664)),
                               ('lda', LinearDiscriminantAnalysis())],
                   final_estimator=LogisticRegression(max_iter=1000,
                                                      random_state=6664))

##**Resultado do modelo**

In [ ]:
try:
  y_pred = int(model.predict(input))

  dic = {0:"Adjuvante",1:"Neoadjuvante"}

  y_pred = dic[y_pred]

  perc = str(model.predict_proba(input)[0][0]) + " para o tratamento Adjuvante e " + str(model.predict_proba(input)[0][1]) + " para o tratamento Neoajduvante"


  print("O modelo sugere o tratamento", y_pred ,"\nAcurácia:",model.score(X,y), "\nProbabilidade dos tratamentos para este caso:", perc)
except:
  print("Valor inserido inválido ou faltando")

O modelo sugere o tratamento Adjuvante 
Acurácia: 0.77420998980632 
Probabilidade dos tratamentos para este caso: 0.5208323421297084 para o tratamento Adjuvante e 0.4791676578702916 para o tratamento Neoajduvante
